**Running on GPU T4**

In [7]:
%%capture
!pip install torch transformers bitsandbytes accelerate sqlparse

In [1]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

In [2]:
torch.cuda.is_available()

True

**Load Model in 4-bit**

In [3]:
device = "cuda" # or "cpu"
model_path = "ibm-granite/granite-8b-code-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True,
    device_map=device ,use_cache=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/46.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
prompt = """

Write a SQL query to answer the following question given the database schema.Please wrap your code answer using ```:

Schema:

CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    Department VARCHAR(50),
    Position VARCHAR(50),
    Salary DECIMAL(10, 2)
);

CREATE TABLE EmployeeDetails (
    EmployeeID INT PRIMARY KEY,
    BirthDate DATE,
    Address VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    ZipCode VARCHAR(20),
    Phone VARCHAR(20),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
);

Question: `{question}`:
```sql
"""

In [5]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
        temperature=0.0,
        top_p=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    # return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)
    return outputs[0].split("```sql")[1].split(";")[0]

**To run a single  query**

In [11]:
import time
t1 = time.perf_counter()
question = "Retrieve all columns for employees with an EmployeeID less than 10."
generated_sql = generate_query(question)
print(sqlparse.format(generated_sql, reindent=True))
t2 = time.perf_counter()
print('time taken to run:',t2-t1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



SELECT *
FROM Employees
WHERE EmployeeID < 10
time taken to run: 2.637418959999877


**We will  be evaluating it on 20 sql questions**

In [40]:
sql_questions = [
    "Retrieve all columns for employees with an EmployeeID less than 10.",
    "Retrieve the first name and salary of employees in the IT department.",
    "Count the number of employees in each department.",
    "Retrieve employees whose salary is greater than $70,000.",
    "Retrieve employees sorted by last name in descending order.",
    "Calculate the average salary of employees.",
    "Retrieve employees birth after January 1st, 1982.",
    "Retrieve employees with the first name starting with 'J'.",
    "Retrieve employees sorted by department and position.",
    "Retrieve employees with no assigned department.",
    "Retrieve employees along with their detailed information (from EmployeeDetails).",
    "Calculate the total number of employees and average salary per department.",
    "Retrieve the employee details who live in cities that have more than one employee, along with the count of employees in those cities.",
    "Retrieve employees with salaries between $60,000 and $80,000, sorted by salary.",
    "Retrieve the top 3 highest paid employees.",
    "Retrieve the department with the highest average salary.",
    "Retrieve employees who are earning more than the average salary of their department.",
    "Retrieve the total number of employees in each city and their average salary.",
    "Retrieve the youngest and oldest employees' details along with their ages.",
    "Retrieve employees who have the same last name as any other employee."
]

In [41]:
len(sql_questions)

20

In [8]:
import time
t1 = time.perf_counter()
model_response=  []

for i in sql_questions:
    response = generate_query(i)
    model_response.append(response)
t2 = time.perf_counter()
print('time taken to execute 20 SQL queries:',t2-t1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


time taken to execute 20 SQL queries: 92.61764750099996


In [16]:
import pandas as pd
ground_truth = [
    "SELECT * FROM Employees WHERE EmployeeID < 10;",
    "SELECT FirstName, Salary FROM Employees WHERE Department = 'IT';",
    "SELECT Department, COUNT(*) AS NumEmployees FROM Employees GROUP BY Department;",
    "SELECT * FROM Employees WHERE Salary > 70000;",
    "SELECT * FROM Employees ORDER BY LastName DESC;",
    "SELECT AVG(Salary) AS AvgSalary FROM Employees;",
    "SELECT * FROM EmployeeDetails WHERE BirthDate > '1982-01-01'",
    "SELECT * FROM Employees WHERE FirstName LIKE 'J%';",
    "SELECT * FROM Employees ORDER BY Department, Position;",
    "SELECT * FROM Employees WHERE Department IS NULL;",
    "SELECT E.*, ED.BirthDate, ED.Address, ED.City, ED.State, ED.ZipCode, ED.Phone FROM Employees E JOIN EmployeeDetails ED ON E.EmployeeID = ED.EmployeeID;",
    "SELECT Department, COUNT(*) AS NumEmployees, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department;",
    "SELECT ed.City, COUNT(ed.EmployeeID) AS EmployeeCount, GROUP_CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeNames FROM EmployeeDetails ed JOIN Employees e ON ed.EmployeeID = e.EmployeeID GROUP BY ed.City HAVING COUNT(ed.EmployeeID) > 1 ORDER BY EmployeeCount DESC;",
    "SELECT * FROM Employees WHERE Salary BETWEEN 60000 AND 80000 ORDER BY Salary;",
    "SELECT * FROM Employees ORDER BY Salary DESC LIMIT 3;",
    "SELECT Department, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department ORDER BY AvgSalary DESC LIMIT 1;",
    "SELECT E.* FROM Employees E JOIN (SELECT Department, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department) AS AvgSalaries ON E.Department = AvgSalaries.Department WHERE E.Salary > AvgSalaries.AvgSalary;",
    "SELECT ED.City, COUNT(*) AS NumEmployees, AVG(E.Salary) AS AvgSalary FROM Employees E JOIN EmployeeDetails ED ON E.EmployeeID = ED.EmployeeID GROUP BY ED.City;",
    "SELECT e.FirstName, e.LastName, ed.BirthDate, TIMESTAMPDIFF(YEAR, ed.BirthDate, CURDATE()) AS Age, CASE WHEN ed.BirthDate = (SELECT MIN(BirthDate) FROM EmployeeDetails) THEN 'Oldest' WHEN ed.BirthDate = (SELECT MAX(BirthDate) FROM EmployeeDetails) THEN 'Youngest' END AS AgeCategory FROM EmployeeDetails ed JOIN Employees e ON ed.EmployeeID = e.EmployeeID WHERE ed.BirthDate = (SELECT MIN(BirthDate) FROM EmployeeDetails) OR ed.BirthDate = (SELECT MAX(BirthDate) FROM EmployeeDetails);",
    "SELECT E1.EmployeeID, E1.FirstName, E1.LastName FROM Employees E1 JOIN Employees E2 ON E1.LastName = E2.LastName AND E1.EmployeeID != E2.EmployeeID;"
]

# Create a DataFrame
df = pd.DataFrame({
    'Question': sql_questions,
    'Ground Truth Query':ground_truth ,
    'Granite Response (LLM)': model_response ,
    'Query Level':['simple','simple','simple','simple','simple','simple','simple','simple','simple','simple','medium','medium','medium','medium','medium','complex','complex','complex','complex','complex'],
    'Match':['correct','correct' ,'correct','correct','correct','correct' ,'correct','correct','correct','correct','correct','correct','wrong','correct','wrong','correct','correct','wrong','wrong','correct']
})

In [17]:
df

,Question,Ground Truth Query,Granite Response (LLM),Query Level,Match
0,Retrieve all columns for employees with an Emp...,SELECT * FROM Employees WHERE EmployeeID < 10;,\nSELECT * FROM Employees WHERE EmployeeID < 10,simple,correct
1,Retrieve the first name and salary of employee...,"SELECT FirstName, Salary FROM Employees WHERE ...","\nSELECT FirstName, Salary\nFROM Employees\nWH...",simple,correct
2,Count the number of employees in each department.,"SELECT Department, COUNT(*) AS NumEmployees FR...","\nSELECT COUNT(*) AS NumberOfEmployees, Depart...",simple,correct
3,Retrieve employees whose salary is greater tha...,SELECT * FROM Employees WHERE Salary > 70000;,\nSELECT *\nFROM Employees\nWHERE Salary > 70000,simple,correct
4,Retrieve employees sorted by last name in desc...,SELECT * FROM Employees ORDER BY LastName DESC;,\nSELECT * FROM Employees ORDER BY LastName DESC,simple,correct
5,Calculate the average salary of employees.,SELECT AVG(Salary) AS AvgSalary FROM Employees;,\nSELECT AVG(Salary) FROM Employees,simple,correct
6,"Retrieve employees birth after January 1st, 1982.",SELECT * FROM EmployeeDetails WHERE BirthDate ...,\nSELECT *\nFROM EmployeeDetails\nWHERE BirthD...,simple,correct
7,Retrieve employees with the first name startin...,SELECT * FROM Employees WHERE FirstName LIKE '...,\nSELECT * FROM Employees WHERE FirstName LIKE...,simple,correct
8,Retrieve employees sorted by department and po...,"SELECT * FROM Employees ORDER BY Department, P...",\nSELECT * FROM Employees\nORDER BY Department...,simple,correct
9,Retrieve employees with no assigned department.,SELECT * FROM Employees WHERE Department IS NULL;,"\nSELECT FirstName, LastName, Email\nFROM Empl...",simple,correct


In [13]:
df.to_csv("granite.csv")

**Time took to execute 20 queries is 92 seconds . Out of 20  questions , 2 are wrong with level as medium and 2 from complex level**

**Now , lets see how much time it will  take to run 20  queries on cpu**

In [1]:
%%capture
!pip install sqlparse

In [3]:
torch.cuda.is_available()

False

In [32]:
# !pip install llama-cpp-python
# !pip install langchain_community

In [33]:
# !wget https://huggingface.co/ibm-granite/granite-8b-code-instruct-GGUF/resolve/main/granite-8b-code-instruct.Q4_K_M.gguf?download=true

In [34]:
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
# GLOBAL VARIABLES
my_model_path = "/content/granite-8b-code-instruct.Q4_K_M.gguf?download=true"
# LOAD THE MODEL
model = LlamaCpp(model_path=my_model_path,temperature=0.0,top_p=1)

llama_model_loader: loaded meta data with 26 key-value pairs and 578 tensors from /content/granite-8b-code-instruct.Q4_K_M.gguf?download=true (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = granite-8b-code-instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 36
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:       

In [35]:
template = """

Write a SQL query to answer the following question given the database schema.Please wrap your code answer using ```:

Schema:

CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    Department VARCHAR(50),
    Position VARCHAR(50),
    Salary DECIMAL(10, 2)
);

CREATE TABLE EmployeeDetails (
    EmployeeID INT PRIMARY KEY,
    BirthDate DATE,
    Address VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    ZipCode VARCHAR(20),
    Phone VARCHAR(20),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
);

Question: `{question}`:
```sql
"""
prompt = PromptTemplate.from_template(template=template)

In [37]:
llm_chain = prompt | model

In [38]:
import sqlparse
response = llm_chain.invoke("Retrieve all columns for employees with an EmployeeID less than 10.")
print(sqlparse.format(response))


llama_print_timings:        load time =    6702.78 ms
llama_print_timings:      sample time =      16.32 ms /    16 runs   (    1.02 ms per token,   980.45 tokens per second)
llama_print_timings: prompt eval time =  141439.03 ms /   192 tokens (  736.66 ms per token,     1.36 tokens per second)
llama_print_timings:        eval time =   12918.83 ms /    15 runs   (  861.26 ms per token,     1.16 tokens per second)
llama_print_timings:       total time =  154430.95 ms /   207 tokens


SELECT * FROM Employees WHERE EmployeeID < 10;
```


In [42]:
import time
t1 = time.perf_counter()
model_response=  []

for i in sql_questions:
    response = llm_chain.invoke(i)
    model_response.append(response)
t2 = time.perf_counter()
print('time taken to execute 20 SQL queries on cpu:',t2-t1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6702.78 ms
llama_print_timings:      sample time =      15.40 ms /    16 runs   (    0.96 ms per token,  1038.76 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   15587.42 ms /    16 runs   (  974.21 ms per token,     1.03 tokens per second)
llama_print_timings:       total time =   15625.60 ms /    16 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6702.78 ms
llama_print_timings:      sample time =      17.64 ms /    18 runs   (    0.98 ms per token,  1020.18 tokens per second)
llama_print_timings: prompt eval time =    8707.22 ms /    16 tokens (  544.20 ms per token,     1.84 tokens per second)
llama_print_timings:        eval time =   15531.01 ms /    18 runs   (  862.83 ms per token,     1.16 tokens per second)
llama_print_timings:       to

time taken to execute 20 SQL queries on cpu: 645.0212495769997


In [44]:
import pandas as pd
ground_truth = [
    "SELECT * FROM Employees WHERE EmployeeID < 10;",
    "SELECT FirstName, Salary FROM Employees WHERE Department = 'IT';",
    "SELECT Department, COUNT(*) AS NumEmployees FROM Employees GROUP BY Department;",
    "SELECT * FROM Employees WHERE Salary > 70000;",
    "SELECT * FROM Employees ORDER BY LastName DESC;",
    "SELECT AVG(Salary) AS AvgSalary FROM Employees;",
    "SELECT * FROM EmployeeDetails WHERE BirthDate > '1982-01-01'",
    "SELECT * FROM Employees WHERE FirstName LIKE 'J%';",
    "SELECT * FROM Employees ORDER BY Department, Position;",
    "SELECT * FROM Employees WHERE Department IS NULL;",
    "SELECT E.*, ED.BirthDate, ED.Address, ED.City, ED.State, ED.ZipCode, ED.Phone FROM Employees E JOIN EmployeeDetails ED ON E.EmployeeID = ED.EmployeeID;",
    "SELECT Department, COUNT(*) AS NumEmployees, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department;",
    "SELECT ed.City, COUNT(ed.EmployeeID) AS EmployeeCount, GROUP_CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeNames FROM EmployeeDetails ed JOIN Employees e ON ed.EmployeeID = e.EmployeeID GROUP BY ed.City HAVING COUNT(ed.EmployeeID) > 1 ORDER BY EmployeeCount DESC;",
    "SELECT * FROM Employees WHERE Salary BETWEEN 60000 AND 80000 ORDER BY Salary;",
    "SELECT * FROM Employees ORDER BY Salary DESC LIMIT 3;",
    "SELECT Department, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department ORDER BY AvgSalary DESC LIMIT 1;",
    "SELECT E.* FROM Employees E JOIN (SELECT Department, AVG(Salary) AS AvgSalary FROM Employees GROUP BY Department) AS AvgSalaries ON E.Department = AvgSalaries.Department WHERE E.Salary > AvgSalaries.AvgSalary;",
    "SELECT ED.City, COUNT(*) AS NumEmployees, AVG(E.Salary) AS AvgSalary FROM Employees E JOIN EmployeeDetails ED ON E.EmployeeID = ED.EmployeeID GROUP BY ED.City;",
    "SELECT e.FirstName, e.LastName, ed.BirthDate, TIMESTAMPDIFF(YEAR, ed.BirthDate, CURDATE()) AS Age, CASE WHEN ed.BirthDate = (SELECT MIN(BirthDate) FROM EmployeeDetails) THEN 'Oldest' WHEN ed.BirthDate = (SELECT MAX(BirthDate) FROM EmployeeDetails) THEN 'Youngest' END AS AgeCategory FROM EmployeeDetails ed JOIN Employees e ON ed.EmployeeID = e.EmployeeID WHERE ed.BirthDate = (SELECT MIN(BirthDate) FROM EmployeeDetails) OR ed.BirthDate = (SELECT MAX(BirthDate) FROM EmployeeDetails);",
    "SELECT E1.EmployeeID, E1.FirstName, E1.LastName FROM Employees E1 JOIN Employees E2 ON E1.LastName = E2.LastName AND E1.EmployeeID != E2.EmployeeID;"
]

# Create a DataFrame
df = pd.DataFrame({
    'Question': sql_questions,
    'Ground Truth Query':ground_truth ,
    'Granite Response (LLM)': model_response ,
    'Query Level':['simple','simple','simple','simple','simple','simple','simple','simple','simple','simple','medium','medium','medium','medium','medium','complex','complex','complex','complex','complex'],
    'Match':['correct','correct' ,'correct','correct','correct','correct' ,'correct','correct','correct','correct','correct','correct','wrong','correct','wrong','correct','correct','wrong','wrong','correct']
})

Exception ignored in: <function Llama.__del__ at 0x7994afd0ac20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py", line 1972, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py", line 1969, in close
    self._stack.close()
AttributeError: 'Llama' object has no attribute '_stack'


In [45]:
df

,Question,Ground Truth Query,Granite Response (LLM),Query Level,Match
0,Retrieve all columns for employees with an Emp...,SELECT * FROM Employees WHERE EmployeeID < 10;,SELECT * FROM Employees WHERE EmployeeID < 10;...,simple,correct
1,Retrieve the first name and salary of employee...,"SELECT FirstName, Salary FROM Employees WHERE ...","SELECT FirstName, Salary FROM Employees WHERE ...",simple,correct
2,Count the number of employees in each department.,"SELECT Department, COUNT(*) AS NumEmployees FR...","SELECT COUNT(*) AS NumberOfEmployees, Departme...",simple,correct
3,Retrieve employees whose salary is greater tha...,SELECT * FROM Employees WHERE Salary > 70000;,SELECT * FROM Employees WHERE Salary > 70000;\...,simple,correct
4,Retrieve employees sorted by last name in desc...,SELECT * FROM Employees ORDER BY LastName DESC;,SELECT * FROM Employees ORDER BY LastName DESC...,simple,correct
5,Calculate the average salary of employees.,SELECT AVG(Salary) AS AvgSalary FROM Employees;,SELECT AVG(Salary) AS AverageSalary FROM Emplo...,simple,correct
6,"Retrieve employees birth after January 1st, 1982.",SELECT * FROM EmployeeDetails WHERE BirthDate ...,SELECT * FROM EmployeeDetails WHERE BirthDate ...,simple,correct
7,Retrieve employees with the first name startin...,SELECT * FROM Employees WHERE FirstName LIKE '...,SELECT * FROM Employees WHERE FirstName LIKE '...,simple,correct
8,Retrieve employees sorted by department and po...,"SELECT * FROM Employees ORDER BY Department, P...","SELECT FirstName, LastName, Department, Positi...",simple,correct
9,Retrieve employees with no assigned department.,SELECT * FROM Employees WHERE Department IS NULL;,"SELECT FirstName, LastName, Email FROM Employe...",simple,correct


**Time took to execute 20 queries on cpu = 645 seconds which is appro. 11 minutes**